In [11]:
!pip install --q unstructured langchain pdfminer.six
!pip install "unstructured[all-docs]"

In [12]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [13]:
local_path="TM_Cyclone_NIDM24.pdf"

if local_path:
  loader=UnstructuredPDFLoader(file_path=local_path)
  data=loader.load()
else:
  print("upload a pdf file")

print("done")

done


In [14]:
!ollama pull nomic-embed-text

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest 
pulling 970aa74c0a90: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 274 MB                         
pulling c71d239df917: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  11 KB                         
pulling ce4a164fc046: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�   17 B                         
pulling 31df23ea7daa: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [15]:
!ollama list
!pip install sentence-transformers faiss-cpu langchain

NAME                       ID              SIZE      MODIFIED               
nomic-embed-text:latest    0a109f422b47    274 MB    Less than a second ago    
llama3.2:latest            a80c4f17acd5    2.0 GB    5 weeks ago               


In [16]:
!pip install --q chromadb
!pip install --q langchain_text_splitters

In [17]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [22]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=7500,chunk_overlap=100)
chunks=text_splitter.split_documents(data)

In [23]:
vector_db=Chroma.from_documents(documents=chunks,
                                embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
                                collection_name="local-rag"
                                )

C:\Users\ojasb\AppData\Local\Temp\ipykernel_8436\1340267426.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████| 28/28 [01:43<00:00,  3.68s/it]


In [24]:
print("done")

done


In [27]:
from langchain.prompts import ChatPromptTemplate,PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [38]:
# local_model="mistral"
# llm=ChatOllama(model=local_model)

from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2")  # must match your local Ollama model


In [39]:
QUERY_PROMPT=PromptTemplate(
    input_variables=["question"],
    template="You are an AI language model assistant. Your task is to generate five different versions of the given user question to retrieve documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of distance-based similarity search. Provide these alternative questions separated by newlines.Original question: {question}""",)

In [40]:
retriever=MultiQueryRetriever.from_llm(vector_db.as_retriever(),llm,prompt=QUERY_PROMPT)


template="""Answer the question based ONLY on the following context:
{context}
Question:{question}"""
prompt=ChatPromptTemplate.from_template(template)

In [41]:
chain=(
    {"context":retriever,"question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [42]:
chain.invoke(input(""))

 what is this about


OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.21s/it]


'This appears to be a book or document related to disaster management and cyclone risk management in India. Specifically, it\'s a "Training Module" on Management of Cyclone Disasters published by the National Institute of Disaster Management (NIDM). The document provides guidance, information, and best practices for managing cyclone disasters, including training materials for facilitators and stakeholders.'